## Simulation for BConv

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sympy.ntheory.residue_ntheory import nthroot_mod
import os
import time
from multiprocessing import Pool

In [2]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

In [3]:
def modinv(a, m):
    '''
    input (a, m)
    a: input
    m: Modulus
    '''
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % m

In [4]:
# Bit-Reverse integer
def bit_reverse(a, n):
    return int(('{:0'+str(n)+'b}').format(a)[::-1],2)

def indexReverse(B, v):
    '''
    B : NTT result in bit-reverse order (BO)
    '''
    n = len(B)
    reversed_indices = [0] * n

    for i in range(n):
        reversed_indices[i] = bit_reverse(i, v) # int(format(i, '0' + str(v) + 'b')[::-1], 2)

    result = [0] * n
    for i in range(n):
        result[reversed_indices[i]] = B[i]

    return result

# # Example usage
# B = [0, 1, 2, 3]
# v = 2

# reversed_B = indexReverse(B, v)
# print(reversed_B)

In [5]:
def tfg(n, q):
    if nthroot_mod(-1,n,q) != None:
        psi = int(nthroot_mod(-1,n,q))
        # print("generate psi table...")
        Y_table = [0] * n  # Start with the first element, which is 1 (psi^0 mod q)
        Y_table[0] = 1
        
        for i in range(1, n):
            Y_table[i] = (psi * Y_table[i-1]) % q
    else:
        print("nthroot_mod not found.")

    return Y_table

In [6]:
def tfg_table(psi, n, q):
    # print("generate psi table...")
    Y_table = [0] * n  # Start with the first element, which is 1 (psi^0 mod q)
    Y_table[0] = 1
    
    for i in range(1, n):
        Y_table[i] = (psi * Y_table[i-1]) % q

    return Y_table

In [7]:
# Cooley-Tukey Butterfly Structure
# A0,A1: input coefficients
# W: twiddle factor
# q: modulus
# B0,B1: output coefficients

# JIT-compiled butterfly
# @njit
def CT_Butterfly(A0,A1,W,q):
    r"""
    A0 -------\--|+|-- B0
               \/
               /\
    A1 --|x|--/--|-|-- B1
    """
    M = (A1 * W) % q

    B0 = (A0 + M) % q
    B1 = (A0 - M) % q

    return B0,B1

In [8]:
# Gentleman-Sandle Butterfly Structure
# A0,A1: input coefficients
# W: twiddle factor
# q: modulus
# B0,B1: output coefficients
def GS_Butterfly(A0,A1,W,q):
    r"""
    A0 --\--|+|------- B0
          \/
          /\
    A1 --/--|-|--|x|-- B1
    """
    M0 = (A0 + A1) % q
    M1 = (A0 - A1) % q

    B0 = M0
    B1 = (M1 * W) % q

    return B0,B1

In [9]:
def DIV2(n,q):
    if (n % 2 == 0):
        n = n >> 1  # Right-shift to divide by 2
    else: # n is odd
        n = (n >> 1) + ((q + 1) >> 1)   # Modular adjustment for odd n
    return n

In [10]:
def GS_BU_DIV2(A0,A1,W,q):
    B0, B1 = GS_Butterfly(A0,A1,W,q)
    return DIV2(B0,q),DIV2(B1,q)

In [11]:
# --- NTT functions ---
def NTT(A, Y_table, q, debug=False):
    '''
    Iterative Radix-2 Cooley-Tukey Number Theoretic Transform (NTT)
    We refer to negacyclic convolution
    A       : Polynomial in coefficient form (COEF form) in normal order
    q       : Modulus
    Y_table : contains TF (2nth root of unity) constants in bit-reverse order, 
    hatA    : bit-reverse order (EVAL form / NTT form)
    '''
    n = len(A)        # n - 1, Degree of polynomial,  which is the highest power of the variable in polynomial
    hatA = A.copy()   # hatA = NTT(A)

    v = math.floor(math.log2(n))    # number of stages

    t = n >> 1
    m = 1  
  
    while m < n:
        for i in range(m):
            idx_omega = m + i
            # print("loop",idx_omega)
            W = Y_table[idx_omega]
            for a_idx in range(i * 2**v, (i * 2**v) + t):
                if debug:
                    print(hatA[a_idx], hatA[a_idx + t])
                hatA[a_idx], hatA[a_idx + t] = CT_Butterfly(hatA[a_idx], hatA[a_idx + t], W, q)
                if debug:
                    print(hatA[a_idx], hatA[a_idx + t])
        if debug:
            print(hatA)

        v -= 1
        t = t//2
        m = m*2

    return hatA

In [12]:
# --- iNTT function ---
def INTT(hatA, Y_table, q):
    '''
    Iterative GS-based Inverse NTT (iNTT) using the Gentleman-Sande (GS) butterfly approach.
    We refer to negacyclic convolution.
    
    hatA    : NTT form of A (Polynomial in evaluation form) in bit-reversed order
    q       : Modulus
    Y_table : Contains inverse TF (2nth root of unity) constants in normal order
    A       : Polynomial in coefficient form (COEF form)
    '''
    a = hatA.copy()  # Copy the NTT(A)
    n = len(hatA)    # Degree of the polynomial
    stages = int(math.log2(n))  # Number of stages in iNTT

    # Perform iNTT
    for stage in range(stages):
        t = 1 << stage  # Calculate t as 2^stage (distance between elements)
        m = n >> (stage + 1)  # Number of groups in the current stage

        for i in range(m):  # Iterate over each group in the current stage
            idx_omega = m + i  # Index for twiddle factor
            W = Y_table[idx_omega]  # Twiddle factor for this group

            # Apply GS Butterfly for each pair in the group
            for j in range(i * t * 2, (i * t * 2) + t):
                a[j], a[j + t] = GS_BU_DIV2(a[j], a[j + t], W, q)

    return a  # Return the resulting polynomial in coefficient form


### INTT

In [13]:
# Reading inverse psi table from file
inv_psi_table = pd.read_csv("psi_inv_partp_df.csv")

In [14]:
# Reading modulus
qp = pd.read_csv("qp.csv")
qp = qp.to_numpy(dtype=object).flatten()  # Flatten the DataFrame to a 1D array


In [15]:
mu = (1 << (48*2)) // qp
mu = np.array(mu, dtype=object)

In [16]:
# Reading data from file
cTilda1_partp = pd.read_csv("cTilda1_partp.csv")
cTilda1_partp = cTilda1_partp.to_numpy(dtype=object)  # Flatten the DataFrame to a 1D array

In [17]:
# INTT in ModUp before BConv
cTilda1_coef = [[] for i in range(len(cTilda1_partp))]
ringDim = len(cTilda1_partp[0])
qp_partp = qp[24:32]

for i in range(len(cTilda1_partp)):
    a = cTilda1_partp[i] # EVAL form
    qi = qp_partp[i]
    psi_inv = inv_psi_table.iloc[i].tolist()  # Convert to list for compatibility with INTT
    result = INTT(a, indexReverse(psi_inv, int(math.log2(len(psi_inv)))), qi)
    cTilda1_coef[i] = result
    
cTilda1_coef = np.array(cTilda1_coef, dtype=object)
pd.DataFrame(cTilda1_coef)

,0,1,2,3,4,5,6,7,8,9,...,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535
0,38405975504760,137191805260920,55793489809391,49582118286039,14812524024336,160675458965191,215151346664701,26636106535304,116900489894584,193963755095157,...,140771923837841,87038789839323,197199743679234,200174826458579,4668213245065,242678592560382,179247914006390,97690931196823,203964154682497,184128864724449
1,139790549492330,203066337012430,98549766417402,156030203205858,97192292898268,57582237934841,198889132427051,198633161944783,250950632225925,151130624655967,...,157407971562396,221323193984652,172443067297712,224506808254023,3656175621056,104070401047790,140932763571743,239465015940422,132914776334263,125000203691191
2,196691935889776,228243763223011,263443705673373,215303679858954,155872045566541,162414157415792,222728674485723,37574572700191,52608686296970,37162477245669,...,42478238338706,110827159400322,150050874542802,237157660741307,173912080528710,151447296231874,264941574190122,172014435636739,143020049714041,64603879205508
3,147910635582017,270217847983928,269040021601170,3619359168811,256327367175691,29813720462200,248450225046653,229394315643897,198362788589068,105309097870192,...,279067356124097,72462412463026,198562323330321,144977143669130,247605331607853,117269223499466,78444290318544,218611164411623,51555107778870,135082406089729
4,168477520685897,179940459500404,137411147913232,5769457608100,191761627003745,226504284605751,55341405863139,222154632832231,134675044581223,200526278258637,...,22157198138519,103196630340642,1847147255358,27174249266397,166653400515482,106174493816582,110027674023402,52586646137713,104391986955336,19561300883068
5,9972510925350,223072527921708,253224877085632,111037918168930,54533032816248,254895880227553,182502632429692,149056839802148,30543611926279,53489877306544,...,54596720736766,196587171347964,218645249421425,263223487447708,161306960786700,268030999853523,149457728250137,90349310233451,255314985414630,249025770649134
6,263081519241097,113895163597604,22379735375437,228136839874620,40761526658348,46747161393306,20278221434473,76483972348947,65195151268779,272750291897459,...,258830102965599,71373257204037,71646410969930,151221631735108,172219289982008,259353920480291,12417972182145,238526183298596,85087258778710,32320658398025
7,247941697128800,188491397711561,136658145405922,46309600522023,157615357303357,108198691231907,181634843384093,105652984830909,267096354149625,259413195313557,...,270150763815914,30600311816821,235519708882617,22384358369148,62725671414581,221488641175960,138387521493517,50043362600478,70490567725193,59807851279015


### BConv

In [18]:
QHatInvModq = pd.read_csv("QHatInvModq_df.csv")
QHatInvModq = QHatInvModq.to_numpy().flatten()  # Flatten the DataFrame to a 1D array

In [19]:
QHatModp = pd.read_csv("QHat_ij_ModDown_df.csv")
QHatModp = QHatModp.to_numpy(dtype=object).flatten()  # Flatten the DataFrame to a 1D array

In [27]:
# CRT Base Conversion
def BConv(ringDim, P, Q, a, QHatInvModq, QHatModp):
    result = [[0] * ringDim for _ in range(len(P))]
    for ri in range(ringDim):
        for i in range(len(P)):
            sigma = [0] * len(P)
            sigma[i] = 0
            for j in range(len(Q)):
                # sigma[i] += ((a[j][ri] * QHatInvModq[j]) % Q[j]) * QHatModp[i*len(Q)+j] % P[i]
                # Compute CRT reconstruction with modular arithmetic
                temp1 = (int(a[j][ri]) * int(QHatInvModq[j])) % Q[j]
                temp2 = (temp1 * int(QHatModp[i * len(Q) + j])) % P[i]
                sigma[i] = (sigma[i] + temp2) % P[i]
            result[i][ri] = sigma[i] % P[i]

    return result

In [21]:
qp_partq = qp[0:24]
qp_partp = qp[24:32]

In [30]:
cTilda1_coef_df = pd.DataFrame(cTilda1_coef)

In [31]:
cTilda1_coef_df.to_csv("cTilda1_coef_df.csv", index=False)
cTilda1_coef = cTilda1_coef_df.to_numpy(dtype=object)

In [32]:
moddown_part1 =  BConv(ringDim, qp_partq, qp_partp, cTilda1_coef, QHatInvModq, QHatModp)

print("BConv result:")
pd.DataFrame(moddown_part1)

BConv result:


,0,1,2,3,4,5,6,7,8,9,...,65526,65527,65528,65529,65530,65531,65532,65533,65534,65535
0,199816095429929,87690448660976,30614351233945,213117221640748,156602324749727,192306912624368,131165833612062,101885421845266,141641274918900,242421505965370,...,180859706815338,121349088930549,120119998765473,67707659620673,82494066135523,7194621033066,39699141440074,88138661271135,236967622183107,17103145763606
1,88978908121039,129866086109529,98252426182667,86148967202821,116651899118949,13250132618266,103898124612617,89502816648881,28423594904686,16494474431913,...,80418711437848,32292398678480,105341335493966,120497818644151,16715680339081,46322132301027,39586157683892,80406553853304,34462400911015,57287453379865
2,70842298684291,64608765372520,101815146778418,24088395067847,99551225606506,122403769823694,57283928276557,101528674533786,15756709198764,86009640109228,...,74727076423512,55159142991565,50632685153586,134135156257333,14473675657045,109858544128827,38415927362787,39052925337717,20720619924876,37658549355384
3,9855858489315,82871626427641,24453197716769,103769093646178,88090249830038,2491014397014,107029746233379,80141754969690,103315671235920,106819486423945,...,91146351948068,34382677597072,26456503777651,2130703839480,126068959388640,49360434434596,75161287794663,26084842923291,86051013525426,33228858701647
4,101767635014241,97500860693215,39595092867891,97767950841627,43899225318620,69590833062455,22749659623622,131968820705029,84964479244815,27658487196124,...,90201508380772,74399880553685,66782075169501,60307370165128,61832871501225,16048747117265,32303099316147,112816459662810,99173068981063,51068387487806
5,886147678142,96610082276530,32769982123218,59476479677805,62492427924107,98625204135327,43167695830078,45972207238735,62417937094070,51885963185590,...,122983352848884,20586929295336,30775652267737,90614305322512,99500705676511,67167602197448,100542592324482,72811132568008,5888477868670,131190181093930
6,106615498081719,36934756102418,83453757400804,38360444724917,70633762510216,103879354536857,79048552914348,126738617166090,94027120876918,139658905349802,...,47256995043253,68030844293606,137467643146736,94997852138731,114554490465556,112595829233044,127887424022141,36924211969123,70140601991225,21065965724750
7,63068754652111,11954771417097,67363770179053,54596514235524,39373508289290,139236909587396,76687975709040,14407985594496,102842229007654,116207472410187,...,117557424829394,112384585188381,77950153371452,57812818690995,23502961131258,34341830690139,62533608877071,26386168357921,48283384754463,106399003050256
8,7843080765166,118713235600815,73381270801257,42441747122923,13077405515874,25788451642988,46527726756901,55246255441629,119613362288265,140672057532100,...,79389395568374,50684520375269,126604325639880,16098717489093,83700506812553,64819187993492,119039725423876,62269541573677,138115750968998,50916082551711
9,77024946876299,135559814960620,58699519083518,70317033450887,38802186046643,13825419529676,69143148974301,48634760597175,117840440453995,72386651135283,...,64639658440168,81251012500986,81048703592894,132747625109719,95557292499236,12163966189686,79717958470731,7915009800010,50332387347514,37838427412869
